# Medical Cost Personal Datasets

Predict how much health insurance premiums cost using different regression models

Dataset Link: https://www.kaggle.com/datasets/mirichoi0218/insurance?resource=download

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import calendar
import optuna

from datetime import datetime, date
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", 10000)

In [2]:
df = pd.read_csv("insurance.csv")
display(df.head(5))
display(df.info())
display(df.describe())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


None

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


# Delete NaN rows

In [3]:
print (df.isna().sum())
#print (df.isnull().sum())
#df.dropna(axis=0, inplace=True) #remove all rows with na's.
#df.reset_index(drop=True)
#print (df.isna().sum())

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


# Typecast the columns to the correct type

In [4]:

# Encode smoker to bool instead of str
def smoker_encode(smoker_str):
    return smoker_str == "yes"
df["smoker"] = df["smoker"].apply(smoker_encode)

# Encode sex to bool instead of str
def sex_encode(sex_str):
    return sex_str == "female"
df["sex"] = df["sex"].apply(sex_encode)

# One hot encode categorical data
df = pd.get_dummies(df)
#df = df.astype({'sex': 'int', 'smoker': 'int', 'region': 'int'},copy=True)

display(df.head(5))
display(df.info())

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,True,27.900,0,True,16884.92400,False,False,False,True
1,18,False,33.770,1,False,1725.55230,False,False,True,False
2,28,False,33.000,3,False,4449.46200,False,False,True,False
3,33,False,22.705,0,False,21984.47061,False,True,False,False
4,32,False,28.880,0,False,3866.85520,False,True,False,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   sex               1338 non-null   bool   
 2   bmi               1338 non-null   float64
 3   children          1338 non-null   int64  
 4   smoker            1338 non-null   bool   
 5   charges           1338 non-null   float64
 6   region_northeast  1338 non-null   bool   
 7   region_northwest  1338 non-null   bool   
 8   region_southeast  1338 non-null   bool   
 9   region_southwest  1338 non-null   bool   
dtypes: bool(6), float64(2), int64(2)
memory usage: 49.8 KB


None

# EDA

In [5]:
df_numerical = df[['age','bmi','children','charges']]
df_categorical = df[['sex','smoker','region_northeast','region_northwest','region_southeast','region_southwest','charges']]

In [ ]:
# Plot correlation between features and target
plt.figure(figsize=(10,10))
sns.heatmap(df_numerical.corr(), cmap="YlGnBu", annot=False)
plt.savefig("NumericalFeatures_vs_InsuranceCost_Correlation.png")

In [ ]:
f, axes = plt.subplots(1, len(df_categorical.columns)-1, figsize=(30, 5))
for idx, feature_name in enumerate(df_categorical.columns):
    if feature_name != "charges":
        g = sns.boxplot(data=df_categorical, x=feature_name, y="charges", hue=feature_name, palette=['r','g'], ax=axes[idx])
        g.set(ylabel=None, title= feature_name);

plt.savefig("CategoricalFeatures_vs_InsuranceCost_BoxPlot.png")

In [ ]:
f, axes = plt.subplots(1, len(df_categorical.columns)-1, figsize=(30, 5))
for idx, feature_name in enumerate(df_categorical.columns):
    if feature_name != "charges":
        g = sns.violinplot(data=df_categorical, x=feature_name, y="charges", inner="points", hue=feature_name, palette=['r','g'],ax=axes[idx])
        g.set(ylabel=None, title= feature_name);

plt.savefig("CategoricalFeatures_vs_InsuranceCost_ViolinPlot.png")

# Prepare data to be fed as input to regression models

In [ ]:
# Get numerical and categorical dataframes without the target
x_numerical = df_numerical.drop(columns=['charges'], inplace=False)
x_categorical = df_categorical.drop(columns=['charges'], inplace=False)
y =  df_numerical[['charges']].values.ravel()

# Scale numerical data and stack all X data
StandardScaler().fit_transform(x_numerical.values)
X = np.hstack((StandardScaler().fit_transform(x_numerical.values), x_categorical.values))

# Partition the data

In [ ]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.1, random_state=0)

# Define objective funcions for hyperparameter tuning

In [ ]:
# Objective function for hyperparameter tuning of DecisionTree
random_state = 42
def objective_dtr(trial, X_train, X_valid, y_train, y_valid):
    params = {
    "criterion": trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse"]),
    "splitter": trial.suggest_categorical("splitter", ["best", "random"]),
    "max_depth": trial.suggest_int("max_depth", 1, 10),
    "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
    "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
    "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),    
    "random_state": random_state,
    }
    
    model = DecisionTreeRegressor(**params).fit(X_train, y_train)
    r2 = model.score(X_valid, y_valid)
    #y_predicted = model.predict(X_valid)
    #rmse = mean_squared_error(y_valid, y_predicted, squared=False)
    
    return r2

In [ ]:
# Objective function for hyperparameter tuning of RandomForest
random_state = 42
def objective_rfr(trial, X_train, X_valid, y_train, y_valid):
    params = {
    "n_estimators": trial.suggest_int("n_estimators", 10, 250, step = 10),
    "max_depth": trial.suggest_int("max_depth", 1, 15),
    "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
    "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
    "criterion": trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse"]),
    "random_state": random_state,
    "n_jobs":-1
    }
    
    model = RandomForestRegressor(**params).fit(X_train, y_train)
    r2 = model.score(X_valid, y_valid)
    #y_predicted = model.predict(X_valid)
    #rmse = mean_squared_error(y_valid, y_predicted, squared=False)
    
    return r2

In [ ]:
# Objective function for hyperparameter tuning of Support Vector Machine
def objective_svr(trial, X_train, X_valid, y_train, y_valid):
    params = {
    "C": trial.suggest_float("C", 1e-10, 1e10, log=True),
    "kernel": trial.suggest_categorical("kernel", ["linear", "rbf", "sigmoid"]),
    "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
    "max_iter": 1000
    }
    
    model = SVR(**params).fit(X_train, y_train)
    r2 = model.score(X_valid, y_valid)
    #y_predicted = model.predict(X_valid)
    #rmse = mean_squared_error(y_valid, y_predicted, squared=False)
    
    return r2

# Train and test the regression models

In [ ]:
optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# DecisionTreeRegressor
study_dtr = optuna.create_study(direction = "maximize")
func = lambda trial: objective_dtr(trial, X_train, X_valid, y_train, y_valid)
study_dtr.optimize(func, n_trials = 100, timeout=600)
model = DecisionTreeRegressor(**study_dtr.best_trial.params).fit(X_train_valid, y_train_valid)
dectree_r2 = model.score(X_test, y_test)

# RandomForestRegressor
study_rfr = optuna.create_study(direction = "maximize")
func = lambda trial: objective_rfr(trial, X_train, X_valid, y_train, y_valid)
study_rfr.optimize(func, n_trials = 100, timeout=600)
model = RandomForestRegressor(**study_rfr.best_trial.params).fit(X_train_valid, y_train_valid)
randfor_r2 = model.score(X_test, y_test)

# SVR
study_svr = optuna.create_study(direction = "maximize")
func = lambda trial: objective_svr(trial, X_train, X_valid, y_train, y_valid)
study_svr.optimize(func, n_trials = 100, timeout=600)
model = SVR(**study_svr.best_trial.params).fit(X_train_valid, y_train_valid)
svr_r2 = model.score(X_test, y_test)

In [ ]:
print("DecisionTreeRegressor R2 =", dectree_r2)
print("RandomForestRegressor Tree R2 =", randfor_r2)
print("SVR Tree R2 =", svr_r2)